Dhananjay Tiwari,
Ph.D. Mechanical Engineering,
Mechanical Engineering Laboratory, 1204
University of Illinois at Urbana Champaign

This file has the code for implementation of FLPO optimization problem using using scipy optimizers

In [20]:
# include libraries and dependencies
import numpy as np # eh! computational python programming and not include this? Can't happen!
from scipy import optimize # optimization module in scipy
from scipy.optimize import minimize # minimize function using various algorithms
from scipy.optimize import LinearConstraint # to define linear constraints

In [59]:
# create FLPO class and define its associated functions
class FLPO():

    # define data members of the class
    N : int # number of vehicles
    M : int # number of nodes
    K : int # number of stages
    uav_id : list # vehicle indices
    node_id : list # facility indices
    stop_id : list # destination id
    node_param : dict # node parameters

    stages : list # FLPO stages, K elements
                  #   Nx1  (M+1)x1  (M+1)x1   (M+1)x1    (M+1)x1       1x1
                  #   stg0   stg1    stg2     stg3 . . . . stgK-2     stgK-1
                  #    0      0        0        0  . . . .  0               
                  #    1      1        1        1  . . . .  1
                  #    2      2        2        2  . . . .  2          dest
                  #    .      .        .        .  . . . .  .
                  #    .      .        .        .  . . . .  .
                  #    .      M-1      M-1      M-1  . . .  M-1
                  #    .      dest    dest     dest  . . .  dest
                  #    N-1    
    
    asn_pb : list # list of list of arrays
                  #  --------------                                                                           --------------
                  # |          stagewise probability associations for a vehicle i = 0,1, . . , N-1                          |
                  # |    -----                                                                     -----                    |
                  # |   |                                                                               |                   |
                  # |   |   stg0 ----> stg1 ----------> stg2 -----> . . . . ----> stg(K-1) -----> stgK  |                   |
                  # |   |     .          .                .     . . . . . . .         .                 |                   |
                  # |   |     .          .                .     . . . . . . .         .                 |                   |
                  # |   |     .          .                .     . . . . . . .         .                 |  . . . . .        |
                  # |   |  1x|stg1|  |stg1|x|stg2|    |stg2|x|stg3|   . . . .    |stg(K-1)|x|stgK|      |                   |
                  # |   |  matrix     matrix            matrix  . . . . . . .      matrix               |                   |
                  # |   |     .          .                .     . . . . . . .         .                 |                   |
                  # |   |     .          .                .     . . . . . . .         .                 |                   |
                  # |   |     .          .                .     . . . . . . .         .                 |                   |
                  # |    -----                                                                     -----                    |
                  #  -------------                                                                            --------------


    def __init__(self, N, M):
        # this function intializes the FLPO structure with assigned values
        
        # UAVs, nodes and stopping state indices
        self.N = N # initialize the number of vehicles
        self.M = M # initialize the number of nodes
        self.K = self.M + 2 # 1 initial stage + #intermediate stages equal to the number of facilities + 1 final stage
        self.uav_id = list(range(N)) # 0 1 2 . . . N-1
        self.node_id = list(range(N, N+M)) # N N+1 . . . N+M-1
        self.stop_id = list(range(N+M, N+M+1)) # N+M
        
        # initialize parameters associated to all the facilities
        self.node_param = dict()
        # self.fac_param['locations'] = np.zeros([M, 2])
        self.node_param['schedules'] = np.zeros([M+1, N]) # for each node, there are n-time parameters corresponding to each vehicle

        # define the structure of the stages and the elements in them
        self.stages = [0 for element in range(self.K)] # initialize list stages. Each 0 to be replaced with a column vector
        for stg in range(self.K):
            if stg == 0:
                # initial stage consisting of nodes
                self.stages[stg] = self.uav_id # Nx1 array
            elif stg == self.K-1:
                # final stage consists of only stopping state
                self.stages[stg] = self.stop_id # 1x1 array
            else:
                # other stages consist of facilities and destination
                self.stages[stg] = self.node_id + self.stop_id # (M+1) x 1 array
        
        # initialize association probabilities
        self.asn_pb = [[0 for element1 in range(self.K-1)] for element2 in range(N)]
        for n in range(N): # iteration over each node in the initial stage
            for stg in range(self.K-1): # iteration goes until the penultimate stage
                next_stage = self.stages[stg+1] # get the next stage nodes
                # initialize probabilities as ones (normalized in the next step)
                if stg == 0:

                    # self.asn_pb[n][stg] = np.ones([1, len(next_stage)]) # uniform distribution
                    self.asn_pb[n][stg] = np.random.rand(1, len(next_stage)) # normal distribution
                    self.asn_pb[n][stg] = self.asn_pb[n][stg]/self.asn_pb[n][stg].sum(axis = 1)[:, None] # normalize
                else:
                    # self.asn_pb[n][stg] = np.ones([len(self.stages[stg]), len(self.stages[stg+1])]) # uniform distribution
                    self.asn_pb[n][stg] = np.random.rand(len(self.stages[stg]), len(self.stages[stg+1])) # normal distribution
                    self.asn_pb[n][stg] = self.asn_pb[n][stg]/self.asn_pb[n][stg].sum(axis = 1)[:, None] # normalize


    # define stage transportation cost for each node        
    def cost_stage_transit(self, n, stg, i, j):
        # this function gives the value of stage transition cost for node n
        # from node i in stage stg to node j in stage stg+1
        # for now the cost is defined based on the facility schedules

        t = self.node_param['schedules'] # a 2D matrix with each row representing a facility, and every column representing a node
        # get the indices of i and j and vehicle n
        id_i = self.stages[stg].index(i)
        id_j = self.stages[stg+1].index(j)
        id_n = self.uav_id.index(n)

        # define the constant for lower and upper time bound penalties respectively
        nu = 100
        eta = 100
        
        if i == j:
            # self-hopping has very high penalty
            cost = 100
        elif i == self.stop_id and j != self.stop_id:
            # vehicle cannot go to a node other than stopping state from the stopping state
            cost = 100
        else:
            # time taken to move from node i to node j by vehicle n in intermediate nodes
            # the cost penalizes vehicles if they take a minimum of 30 second and a maximum of 80 second
            # to traverse between the nodes
            cost = t[id_j, id_n] - t[id_i, id_n] + \
                 np.exp(nu * (30 - (t[id_j, id_n] - t[id_i, id_n]))) + \
                    np.exp(eta * ((t[id_j, id_n] - t[id_i, id_n]) - 80))

        return cost
    

    # define the distortion function of a node 
    def distortion(self, n): 
        # this function computes the distortion function of a node n using probability associations and parameters of facilities for the corresponding vehicle
        if n not in self.uav_id: # check if the vehicle argument is valid or not
            raise('invalid node id')
        else:
            # initialize the distortion data structure (same as stages data structure)
            D = [np.zeros(1)]
            D = D + [np.zeros(len(self.stages[stg])) for stg in range(1,self.K)]
            print('\n-- Distortion Matrix --\n')
            for row in D:
                print(row)

            # iterate over stages : final stage --> initial stage
            stg_ids = list(range(self.K))
            stg_ids_rev = stg_ids[::-1]
            for stg in stg_ids_rev:
                print('stage -- ' , stg)
                # iterate over each element of the chosen stage
                if stg > 0:
                    for i in self.stages[stg]:
                        id_i = self.stages[stg].index(i)
                        print('i -- ' , i, id_i)
                        D[stg][id_i] = 0
                        if stg == self.K-1:
                            D[stg][id_i] = 0
                        else:
                            print('next stage nodes -- ')
                            for j in self.stages[stg+1]:
                                id_j = self.stages[stg+1].index(j)
                                print(j , id_j, D[id_i][id_j])
                                D[stg][id_i] = D[stg][id_i] + self.asn_pb[n][stg][id_i, id_j] *\
                                    (D[stg+1][id_j] + self.cost_stage_transit(n, stg, i, j))
                elif stg == 0:
                    i = n
                    id_i = 0
                    print('i -- ' , i , id_i)
                    print('next stage nodes -- ')
                    for j in self.stages[stg+1]:
                        id_j = self.stages[stg+1].index(j)
                        print(j, id_j, D[id_i][id_j])
                        D[stg][id_i] = D[stg][id_j] + self.asn_pb[n][stg][id_i, id_j]*\
                            (D[stg+1][id_j] + self.cost_stage_transit(n, stg, i, j))
        return D
    

    def entropy(self, n): # depends on probability associations
        # ToDo
        return 0


    def free_energy():
        # ToDo
        return 0


    def flat_asn_pb(self, asn_pb):
        # this function unwraps the association probabilities into a column vector

        vec = []
        # unwrap for a vehicle, and then put them all together
        for uav_pb in asn_pb:
            for stg_pb in uav_pb:
                vec = vec + list(stg_pb.flatten())

        return vec


    def wrap_flat_asn_pb(self, vec_asn_pb):
        # this function wraps the association probabilities into their original defined structure inside __init__() function
        # vec_asn_pb (list) should contain probability association corresponding to all the vehicles otherwise the function raises error

        # initialize the required structure of association probability we need
        asn_pb = [[0 for element1 in range(self.K-1)] for element2 in range(self.N)]

        # get the dimension of probability association for a single vehicle
        dim_uav = 0
        # iterate over the stages to compute the above required dimension
        for stg in range(self.K-1):
            if stg == 0:
                dim_uav = dim_uav + len(self.stages[stg+1])
            else:
                dim_uav = dim_uav + len(self.stages[stg])*len(self.stages[stg+1])
        
        # dim_uav times the number of vehicles should be dim of vec_asn_pb
        if dim_uav * len(self.uav_id) != int(len(vec_asn_pb)):
            raise('invalid dimension of association probability vector')
        else:
            for uav in self.uav_id:
                # get the index of uav
                n = self.uav_id.index(uav)
                # slice the probability vector for the corresponding vehicle
                sliced_vec = vec_asn_pb[n*dim_uav : (n+1)*dim_uav]
                k = 0
                for stg in range(self.K-1):
                    if stg == 0:
                        asn_pb[n][stg] = np.reshape( sliced_vec[k : k + 1*len(self.stages[stg+1])] , [1, len(self.stages[stg+1])] )
                        k = k + 1*len(self.stages[stg+1])
                    else:
                        asn_pb[n][stg] = np.reshape( sliced_vec[k : k + len(self.stages[stg])*len(self.stages[stg+1])] , [len(self.stages[stg]), len(self.stages[stg+1])] )
                        k = k + len(self.stages[stg])*len(self.stages[stg+1])
        
        return asn_pb


    def flat_schedules(self, schedules):
        # this converts the original data structure of schedules (see __init__() function) into a 1D array
        return list(schedules.flatten())


    def wrap_flat_schedules(self, vec_schedules):
        # this converts the 1D array of schedules into its original defined structure (see __init__() function)
        
        # first check if the dimension is correct
        dim = len(vec_schedules)
        if dim != (len(self.node_id) + len(self.stop_id)) * len(self.uav_id):
            raise('invalid dimension of the parameters for time-schedule')

        return np.reshape(vec_schedules, [len(self.node_id)+len(self.stop_id), len(self.uav_id)])


    def cons_asn_pb(self, vec_asn_pb):
        # this function constraints the distributions that the sum of probabilities at every node is 1 for every vehicle
        # input : consists of all the probability associations in the vector form
        # output : consists of linear constraints as required by scipy.optimize.minimize function

        # get the dimension of probability association for a single vehicle
        dim_uav = 0
        # iterate over the stages to compute the above required dimension
        for stg in range(self.K-1):
            if stg == 0: # in the zeroth stage only 1 vehicle is considered and multiplied with the dimension of the stage 1
                dim_uav = dim_uav + 1 * len(self.stages[stg+1])
            else: # otherwise the original dimensions of the current stage is multiplied with that the next one
                dim_uav = dim_uav + len(self.stages[stg]) * len(self.stages[stg+1])
        
        # dim_uav times the number of vehicles should be dim of vec_asn_pb
        if dim_uav * len(self.uav_id) != int(len(vec_asn_pb)):
            raise('invalid dimension of association probability vector')

        # the number of equality constraints is equal to the sum of cardinality of every stage (except the first and the final stage)
        num_eq_cons = 0
        # iterate over the vehicles
        for n in range(self.N):
            # iterate till the penultimate stage
            for stg in range(self.K-1):
                if stg == 0: # in the initial stage, only vehicle n is counted
                    num_eq_cons = num_eq_cons + 1
                else: # otherwise count all the nodes in a stage
                    num_eq_cons = num_eq_cons + len(self.stages[stg])

        # initialize the linear constraint matrix
        fat_mat = [[0 for i in range(len(vec_asn_pb))] for j in range(num_eq_cons)]
        # initialize counters
        row_count = 0 # to trace rows of fat_mat
        col_count = 0 # to trace cols of fat_mat
        # iterate over vehicles
        for n in range(self.N):
            # corresponding to the first transition of a vehicle
            # iterate over stages from 1 to K-1
            for stg in range(self.K-1):
                if stg == 0:
                    # iteration over stage 1 for columns of fat_mat in the first row
                    for j in range(len(self.stages[stg+1])):
                        fat_mat[row_count][col_count] = 1
                        col_count = col_count + 1 # update column count for the next column
                    row_count = row_count + 1 # update row count for the next row
                else:
                    # iterate over the rows that correspond to the current stage
                    for i in range(len(self.stages[stg])):
                        # iterate over the that cols correspond to the next stage
                        for j in range(len(self.stages[stg+1])):
                            fat_mat[row_count][col_count] = 1
                            col_count = col_count + 1 # update column count for the next column
                        row_count = row_count + 1 # update row count for the next row
            
        # define the lower bound for linear inequality/equality constraints
        lb = list(np.ones(num_eq_cons)) # all probability distributions sum to 1
        # define the upper bound for linear inquality/equality constraints (equal to lb in case of equality)
        ub = list(np.ones(num_eq_cons)) # all probability distributions sum to 1

        # define the linear constraint in the form needed for scicpy.optimize.minimize
        lin_con = LinearConstraint(fat_mat, lb, ub) 

        return lin_con
    

    def objective(self, x):
        # this function computes the objective of the FLPO problem from distortion of each vehicle and entropy
        # of probability associations for each vehicle
        # input : x = [ flat_asn_pb , time_schedules ] is a vector consisting of all the probability associations and node time schedule parameters

        asn_pb = x # SLICE IT
        params = x # SLICE IT

        # assign the association probabilities and node parameters 
        self.asn_pb = asn_pb
        self.node_param = params

        # initialize the total cost
        cost = 0

        # initialize the weights given to each vehicle
        uav_weights = np.ones(len(self.uav_id))/len(self.uav_id) # uniform distribution for now

        # compute distortion cost by performing weighted sum over each vehicle
        for n in self.uav_id:
            id_n = self.uav_id.index(n)
            distortion_cost = distortion_cost + uav_weights[id_n] * self.distortion(n)

        # ToDo : compute entropy cost by performing weighted sum over each vehicle
        
        cost = distortion_cost # + entropy_cost
        return cost



Minimize the distortion of FLPO with respect to probability associations and node schedule parameters

In [60]:
# define functions
def printMatrix(mat , mat_name):
    print(mat_name)
    for row in mat:
        print(row , '\n')
    return 0

# initialize variables
N = 2 # number of vehicles
M = 3 # number of nodes

# create an instance of FLPO class
flpo = FLPO(N, M) 

flpo.distortion(0)
# print variales
# print(' -- Stages --\n' , flpo.stages)
# print(' \n-- Association probabilities --\n' , flpo.asn_pb)
# vec_asn_pb = flpo.flat_asn_pb(flpo.asn_pb)
# print(' \n-- Flattened Association Probabilities --\n ' , vec_asn_pb)
# print(' \n-- Wrapped Association Probailities --\n ' , flpo.wrap_flat_asn_pb(vec_asn_pb))

# print(' \n-- Time schedules --\n ' , flpo.node_param['schedules'])
# flat_time_schedules = flpo.flat_schedules(flpo.node_param['schedules'])
# print(' \n-- Flattened time schedules --\n' , flat_time_schedules)
# print(' \n-- Wrapped time schedules --\n' , flpo.wrap_flat_schedules(flat_time_schedules))

# fat_mat = flpo.cons_asn_pb(vec_asn_pb)
# printMatrix(fat_mat , 'Probability associations constraint matrix')

# vec_schedules = flpo.flat_schedules(flpo.node_param['schedules'])

# res = minimize(flpo.objective, init_condition)


-- Distortion Matrix --

[0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0.]
stage --  4
i --  5 0
stage --  3
i --  2 0
next stage nodes -- 
5 0 0.0
i --  3 1
next stage nodes -- 
5 0 0.0
i --  4 2
next stage nodes -- 
5 0 0.0
i --  5 3
next stage nodes -- 
5 0 inf
stage --  2
i --  2 0
next stage nodes -- 
2 0 0.0


/var/folders/sh/ndxykdmj1q9cwr2kpvkvcbs80000gn/T/ipykernel_67614/2141807397.py:116: RuntimeWarning: overflow encountered in exp
  np.exp(nu * (30 - (t[id_j, id_n] - t[id_i, id_n]))) + \


IndexError: index 1 is out of bounds for axis 0 with size 1